In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
df=pd.read_csv('classification_train.csv')

In [2]:
array=df.to_numpy()
x=array[:24000,2:]
y=array[:,1]
y_logistic=np.zeros((24000,10))
for i in range(24000):
    y_logistic[i,y[i]]=1

In [3]:
def feature_scaling(x):
    shape=x.shape
    m=shape[0]
    if len(shape)==1:
        n=1
    else:
        n=shape[1]
    for i in range(n):
        sigma_x=np.sum(x[0:,i])
        mean_x=sigma_x/m
        deviation=(x[0:,i]-mean_x)**2
        Std_D=np.sum(deviation)
        Std_D=Std_D/m
        Std_D=Std_D**(.5)
        """Std_D=np.std(x[0:,i], dtype = np.float64)"""
        x[0:,i]=x[0:,i]-mean_x
        x[0:,i]=x[0:,i]/Std_D
    return x
x=feature_scaling(x)

In [4]:
##n is an array of no of neurons in all layers except input layer
def layer_formation(n):
    nf=np.insert(n,0,784)
    #print(nf.shape)
    l=len(nf)
    w={}
    b={}
    for i in range (l-1):
        np.random.seed(42)
        w[i+1]=np.random.randn(nf[i+1],nf[i])/10
        b[i+1]=np.random.randn(nf[i+1])/10
    return w,b    

In [5]:
def z_function(x,w,b):
    #bnew=np.tile(b,(30000,1))
    z=(np.matmul(x,w.T)+b)
    return z

In [6]:
def RelU_(z):
    relu=np.maximum(0,z)
    return relu
RelU=np.vectorize(RelU_)

In [7]:
def Tanh_function(z):
    return np.tanh(z)
def Tanh_function_d(z):
    return 1-(np.tanh(z))**2

In [8]:
def Hits(y_predicted,y):
    shape=y.shape
    m=shape[0]
    #print(m)
    count=0
    for i in range (m):
        if y_predicted[i]==y[i]:
            count+=1
    accuracy=(count/m)*100
    return accuracy,count

In [9]:
def RelU_d_(z):
    return z>0
RelU_d=np.vectorize(RelU_d_)

In [10]:
 def sigmoid_(z):
        return 1/(1+np.exp(-z))
sigmoid=np.vectorize(sigmoid_)

In [11]:
def sigmoid_d_(z):
    s=sigmoid_(z)
    d=s*(1-s)
    return d
sigmoid_d=np.vectorize(sigmoid_d_)

In [12]:
def forward_prop(x,w,b,activation):
    l=len(w)
    a={}
    a[0]=x
    if activation=="tanh":
        for i in range (l-1):
        a[i+1]=Tanh_function(z_function(a[i],w[i+1],b[i+1]))
    elif activation=="relu":
        for i in range (l-1):
        a[i+1]=RelU(z_function(a[i],w[i+1],b[i+1])) 
    a[l]=softmax(a[l-1],w[l],b[l])
    return a

In [13]:
def cost_function(x,y,w,b):
    l=len(w)
    a=forward_prop(x,w,b)
    cost=np.sum(y*np.log(a[l]))
    return -cost
    

In [14]:

def back_prop(x,y,w,b,activation):
    m=x.shape[0]
    l=len(w)
    a=forward_prop(x,w,b,activation)
    dw={}
    db={}
    dc_z=(1/m)*(a[l]-y)
    for i in range (l):
        dw[l-i]=np.matmul(dc_z.T,a[l-i-1])
        db[l-i]=np.sum(dc_z,axis=0)
        if l-i-2<0:
            break
        if activation=="tanh":
            dc_z=np.matmul(dc_z,w[l-i])*Tanh_function_d(z_function(a[l-i-2],w[l-i-1],b[l-i-1]))
        elif activation=="relu":
            dc_z=np.matmul(dc_z,w[l-i])*RelU_d(z_function(a[l-i-2],w[l-i-1],b[l-i-1]))

        
    return dw,db
    

In [15]:
def update_parameters(x,y,w,b,alpha,iterations,activation):
    m=x.shape[0]
    w_new=w
    b_new=b
    l=len(w)
    y_l=np.argmax(y,axis=1)
    for i in range (iterations):
        dw,db=back_prop(x,y,w_new,b_new,activation)
        if i%30==0:
            afinal=forward_prop(x,w_new,b_new,activation)
            y_final=np.zeros(m)
            for j in range (m):
                y_final[j]=np.argmax(afinal[2][j])
            print(f"accuracy after {i} iterations is {Hits(y_final,y_l)}")
            
        for j in range (l):
                w_new[j+1]=w_new[j+1]-alpha*dw[j+1]
                b_new[j+1]=b_new[j+1]-alpha*db[j+1]
    return w_new,b_new

In [16]:
def softmax(x,w,b):
    z=z_function(x,w,b)
    z=z-(np.max(z, axis=1)).reshape(-1,1)
    #print(z.shape)
    softmax=(np.exp(z).T/np.sum(np.exp(z),axis=1)).T
    return softmax

In [17]:
n=np.array([10,10])
w,b=layer_formation(n)

In [19]:
 w_new,b_new=update_parameters(x,y_logistic,w,b,0.01,1500,"relu")

accuracy after 0 iterations is (6.808333333333333, 1634)
accuracy after 30 iterations is (11.504166666666666, 2761)
accuracy after 60 iterations is (19.820833333333333, 4757)
accuracy after 90 iterations is (25.016666666666666, 6004)
accuracy after 120 iterations is (28.695833333333333, 6887)
accuracy after 150 iterations is (33.49583333333334, 8039)
accuracy after 180 iterations is (38.32083333333333, 9197)
accuracy after 210 iterations is (42.7375, 10257)
accuracy after 240 iterations is (47.333333333333336, 11360)
accuracy after 270 iterations is (51.24999999999999, 12300)
accuracy after 300 iterations is (54.07083333333333, 12977)
accuracy after 330 iterations is (56.108333333333334, 13466)
accuracy after 360 iterations is (57.604166666666664, 13825)
accuracy after 390 iterations is (58.916666666666664, 14140)
accuracy after 420 iterations is (60.24583333333333, 14459)
accuracy after 450 iterations is (61.64166666666666, 14794)
accuracy after 480 iterations is (62.87083333333333, 1

In [20]:
 w_new1,b_new1=update_parameters(x,y_logistic,w_new,b_new,0.01,500,"relu")

accuracy after 0 iterations is (74.35416666666667, 17845)
accuracy after 30 iterations is (74.425, 17862)
accuracy after 60 iterations is (74.49583333333332, 17879)
accuracy after 90 iterations is (74.58749999999999, 17901)
accuracy after 120 iterations is (74.72083333333333, 17933)
accuracy after 150 iterations is (74.8, 17952)
accuracy after 180 iterations is (74.91666666666667, 17980)
accuracy after 210 iterations is (74.95833333333334, 17990)
accuracy after 240 iterations is (75.00833333333333, 18002)
accuracy after 270 iterations is (75.09583333333333, 18023)
accuracy after 300 iterations is (75.17083333333333, 18041)
accuracy after 330 iterations is (75.24583333333334, 18059)
accuracy after 360 iterations is (75.32083333333334, 18077)
accuracy after 390 iterations is (75.36666666666667, 18088)
accuracy after 420 iterations is (75.47083333333333, 18113)
accuracy after 450 iterations is (75.575, 18138)
accuracy after 480 iterations is (75.625, 18150)
